In [13]:
import pylab as pl
import sympy as sp
from mtm026 import *

P0 = 1          #[N]
L1 = 5          #[m]
L2 = 4.5        #[m]
a = 0.05        #[m]
t = 0.002       #[m]
E = 200e9       #[Pa]
v = 0.3

I_1 = 1.4771e-7
I_2 = 1.4771e-7

K = np.zeros([16, 16])
K_sig = np.zeros([16, 16])
f = np.zeros([16, 1])

ke1 = Ke_balk(E*I_1, L2)
ke5 = Ke_balk(E*I_2, L1)
ke2 = ke1
ke4 = ke1
ke3 = ke1
ke6 = ke5

ke1_sig = Ke_sigma_balk(P0, L2)
ke5_sig = Ke_sigma_balk(P0, L1)
ke2_sig = ke1_sig
ke4_sig = ke1_sig
ke3_sig = ke1_sig
ke6_sig = ke5_sig

f[15] = -P0

assem(K, ke1, dofs=[1,2,3,4])
assem(K, ke2, dofs=[3,4,5,6])
assem(K, ke3, dofs=[7,8,9,10])
assem(K, ke4, dofs=[9,10,11,12])
assem(K, ke5, dofs=[3,4,9,10])
assem(K, ke6, dofs=[5,6,11,12])

assem(K_sig, ke1_sig, dofs=[1,2,3,4])
assem(K_sig, ke2_sig, dofs=[3,4,5,6])
assem(K_sig, ke3_sig, dofs=[7,8,9,10])
assem(K_sig, ke4_sig, dofs=[9,10,11,12])
assem(K_sig, ke5_sig, dofs=[3,4,9,10])
assem(K_sig, ke6_sig, dofs=[5,6,11,12])

displayvar("K", K)
displayvar("K_\sigma", K_sig)
displayvar("f", f)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [14]:
a9 = a3
a12 = a6

# Definiera våra fria frihetsgrader
free_dofs = [3,4,5,6,7,8,10,12]

# Reducera till enbart fria frihetsgrader
K_red = extract_block(K, free_dofs, free_dofs)
K_sigma_red = extract_block(K_sig, free_dofs, free_dofs)

In [15]:
# Lös det generaliserade egenvärdesproblemet och få ut hur många gånger
# större krafter måste vara (varje λ) samt hur den knäckformen ser ut (varje kolonn i a ger en ny form)
λ, a = eig(K_red, K_sigma_red)

# Sortera för att få den lägsta knäckkraften först
sortingOrder = np.argsort(λ)
λ = λ[sortingOrder]

# Sortera formerna på samma sätt
a = a[:, sortingOrder]


displayvar("λ_1",np.real(λ[0]))
displayvar("a_1",a[:,0])

<IPython.core.display.Math object>

<IPython.core.display.Math object>